In [62]:
from sklearn.metrics.pairwise import cosine_similarity
from litQeval.eval_utils import get_data, evaluate_recall
import pandas as pd
import numpy as np

In [63]:
baseline = '"Cervical Myelopathy"'
predicted = """
"spinal stenosis" OR "spinal surgery risks" OR "cervical myelopathy" OR "gait disturbance" OR "corticosteroid injections" OR "degenerative disc disease" OR "surgery for myelopathy" OR "myelopathy symptoms" OR "nerve root compression" OR "upper limb weakness" OR "mri cervical spine" OR "cervical radiculopathy" OR "cervical spondylotic myelopathy" OR "myelopathy assessment" OR 
(("rehabilitation" OR "long-term outcomes" OR "patient education" OR "pain management" OR "chronic pain" OR "physical therapy" OR "neurological symptoms" OR "postoperative care" OR "cervical spine" OR "neck pain" OR "reflex changes" OR "mobility issues" OR "spinal cord compression" OR "electromyography (emg)") AND ("Cervical Myelopathy"))
"""

data = get_data(baseline, predicted)
core_pubs = data["core_pubs"]
core_mean_embedding = data["core_mean_embedding"]
baseline_pubs = data["baseline_pubs"]
predicted_pubs = data["predicted_pubs"]
baseline_vs = data["baseline_vs"]
predicted_vs = data["predicted_vs"]
predicted_embeddings = np.array([embedding for embedding in predicted_vs.get(include=["embeddings"])["embeddings"]])
baseline_embeddings = np.array([embedding for embedding in baseline_vs.get(include=["embeddings"])["embeddings"]])

In [64]:
evaluate_recall(core_pubs, baseline_pubs, predicted_pubs)

{'baseline_recall': 0.2765957446808511, 'predicted_recall': 0.723404255319149}

In [65]:
core_mean_embedding.reshape(1, -1).shape, predicted_embeddings.shape, baseline_embeddings.shape

((1, 1536), (37068, 1536), (4298, 1536))

In [72]:
cosine_sim = cosine_similarity(core_mean_embedding, predicted_embeddings).flatten()
predicted_pubs["similarity"] = cosine_sim

threshold = predicted_pubs[predicted_pubs["id"].isin(core_pubs)]["similarity"].min()
relevant_pubs = predicted_pubs[predicted_pubs["similarity"] >= threshold].copy()
relevant_pubs.shape

(11719, 4)

In [76]:
# every element that is more similar than the least similar core publication is considered relevant
relevant_pubs.shape[0] / predicted_pubs.shape[0]

0.3161486996870616